# Negative sentiment data
I selected 100 words from collection of drug name/ slang. \
We have 100 drug related senteces. \
I augmented the 100 sentences by first translating them to hindi and then tralating these sentnces back to english, and applying sunonyms for 2 words in the final sentence.\
Once we have 100 drug words/slangs, and 200 drug related sentences, I replaced drug names in each sentence with all the drug name/ slangs in the dictionary.\
Thus I now have **10000 drug related negative sentiment sentences** for which model would know the importance of sentence as well as the occurance of word.\
In order to prevent our model from identifying the occurance of sentences as the source of negative sentence, I replaced all the drug names with nouns to give positive sentiment with same sentence in absence of drugs/slangs. This gives us **3500 positive sentiment sentences**.
But we also have sentences where the sentiment is positive even if there is a mention of drug name/ slang. I took 25 such sentences and augmented them (as I did for 100 negative sentences) to get 50 sentences and fed in different drug names/slang. This gives us **5000 drug related positive sentiment sentences**.\
We take **1500 positive sentiment sentences** from amazon review dataset on amazon as is.


In [1]:
import numpy as np

dic= np.genfromtxt('./test.txt',dtype='str')
len(dic)

237

In [3]:
dic_list= np.random.choice(dic, size=93, replace=False) #choosing 200 words
final_list= [(x.replace('-',' ')).lower() for x in list(dic_list)]
final_dic= final_list + ['pikachu', 'drugs', 'weed', 'love','meth', 'crystal', 'methamphetamine'] #adding a few more words related to drugs to expand vocabulary
(final_dic) #100 drug name/slang
#alternatively, we could also make sure that all the drug names are present and then choose from the available drug slang.

['super acid',
 'boomers',
 'train',
 'tussin',
 'tweeker',
 'god’s flesh',
 'skippy',
 'grass',
 'crank',
 'sugar',
 'spice',
 'downie',
 'hug drug',
 'duster',
 'lady bub bles',
 'ivory fresh',
 'skittles',
 'white dove',
 'peanut',
 'rainbows',
 'gravel',
 'stackers',
 'musk',
 'poppers',
 'arctic blast',
 'ts and rs',
 'arnolds',
 'chinese tobacco',
 'ts and ritz',
 'great  bear',
 'bean',
 'junk',
 'shot',
 'slick superspeed',
 'hardball',
 'goob',
 'detroit pink',
 'goodfellas',
 'vitamin  d',
 'happy dust',
 'kit kat',
 'snow leopard',
 'fort dodge',
 'bump',
 'snow',
 'coke',
 'green',
 'dihydrolone',
 'tac',
 'pumpers',
 'magic mushrooms',
 'stum bler',
 'mama coca',
 'caps',
 'antifreeze',
 'reynolds',
 'herb',
 'dabbing',
 'twenty five',
 'cadillac express',
 'liquid x',
 'skag',
 'china town',
 'china girl',
 'crank',
 'superman',
 'little smoke',
 'honey oil',
 'beat',
 'cat killer',
 'jelly',
 'belladonna',
 'dabs',
 'shrooms',
 'snappers',
 'liquid  ecstasy',
 'go fast',

Augmenting dataset by changing sentences:

In [4]:
#importing 100 drug related sentences:
import pandas as pd
df1 = pd.read_excel(r'.\dialogou.xlsx')
df2 = pd.read_excel(r'.\dialogou_n.xlsx')
df3 = pd.read_excel(r'.\output.xlsx')
df11=df1[df1['sentiment']==0][0].values[:]
df22=df2[df2['sentiment']==0][0].values[:]
df33=df3[df3['sentiment']==0][0].values[:]
df= np.hstack((df11,df22[:-2],df33)) 
len(df) #100 drug related sentences

100

In [5]:
import re
#replacing all the drug words with 'meth' for simplicity and checking 

pattern = '\\b(' + '|'.join(list(dic) + ['pikachu', 'drugs', 'weed', 'love','meth', 'crystal', 'methamphetamine']) + ')\\b'
replace_with = 'meth'

for i, sentence in enumerate(df):
    df[i] = re.sub(pattern, replace_with, sentence, flags=re.IGNORECASE)

print(len(df))

100


Now augmenting these 100 sentences 

In [6]:
from googletrans import Translator
import random
from textblob import TextBlob

dataset = (df)

# Translation  to augmented dataset
synonym_dataset = []
for sentence in dataset:
    
    #english to hindi
    blob = TextBlob(sentence).translate(from_lang='en', to='hi')
    translated= blob.translate(from_lang='hi', to='en')
    
    synonym_dataset.append(translated.strip())

# Remove duplicates from augmented dataset
final_dataset = list(set(synonym_dataset))

# Print final dataset
print(final_dataset)


[TextBlob("Raw Meth"), TextBlob("I met methy every night during the freshman year"), TextBlob("Maybe we all need meth"), TextBlob("We will never get this chance"), TextBlob("Like meth needle and cold day"), TextBlob("We have to spread meth"), TextBlob("I am dreaming of a Meth Christmas. Today n or gunpowder receives any delivery tomoro and is not available till 10 00 pm"), TextBlob("I saw the dirt that this is pure meth that you have received is the right product"), TextBlob("All Meth going to California"), TextBlob("Please give me some florida meth"), TextBlob("I sometimes like the party very much, but that stuff is really addictive,"), TextBlob("I find the person who was these meths and made a deal"), TextBlob("A packet of Meth will be the perfect gift for my birthday"), TextBlob("Look, there is some fame in that boat."), TextBlob("Keeping an eye on your blue meth"), TextBlob("Smelly"), TextBlob("Do you know that I am a friend who buys you rightly?"), TextBlob("My customer will suppl

Observing this, it is apparent that a few sentences have been changed without loosing the meaning but for a few, textblob gave poor results, so I chose to do only synonyms augmentation.

In [8]:
import nltk
nltk.download('wordnet', "nltk_data/")

nltk.download('omw-1.4', "nltk_data/")

nltk.data.path.append('nltk_data/')
nltk.download('stopwords')

from nltk.corpus import stopwords

from nltk.corpus import wordnet




# function to get list of words after removing stop words
def remove_stopwords(text):
    #"nlp" Object is used to create documents with linguistic annotations.
    stop_words = set(stopwords.words('english'))

    filtered_sentence = [word for word in text.split() if word.lower() not in stop_words]
    filtered_sentence = ' '.join(filtered_sentence)
    #print(token_list)
    #print(filtered_sentence)
    return filtered_sentence

# random insert function
def random_insert(sentence):
    # n is the number of words to replace in sentence

    # get the list of words excluding stop words
    sentence_ = remove_stopwords(sentence)

    words = nltk.word_tokenize(sentence_)
    new_sentence = ''
    
    repeated_words = [] 
    replace_words = []
    # number of words changed depends on the number of words in sentence 40% are changed
    n= int(len(words)*0.4)
    for i in range (n):
    # choose a random word
        word = random.choice(words)

        # if word has already been choosen before move to next word
        if word in repeated_words:
            continue
        if word ==('meth'):
            continue
            
        repeated_words.append (word)

        # get closest synonym of the word
        new_synonym = wordnet.synsets(word)
        if new_synonym:
            new_word = new_synonym[0].lemmas()[0].name()
            if new_word != word:
                
                # maintain a list of words and their synonyms to be replaced
                replace_words.append (new_word)

                # replace the synonym with the word in the sentence
                
                sentence = sentence.replace(word, new_word)
    sentence= ''.join(sentence)
    replace_words= ' '.join(replace_words)
    return sentence, replace_words

synonym_dataset = []
word_list=[]

for sentence in df:
    translated, words_rep= random_insert(sentence.lower())
    
    word_list.append(words_rep.strip())
    synonym_dataset.append(translated.strip())

# Remove duplicates from augmented dataset
final_dataset = list(set(synonym_dataset))

# Print final dataset
print(final_dataset)
negative_200= np.hstack((final_dataset,df)) 

[nltk_data] Downloading package wordnet to nltk_data/...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to nltk_data/...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ddave\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['yeh mate thats cool the meth is so nice though mate amnesia heterosexual from the dam', 'raw uracilncooked meth shard so uracil can do with it how uracil please limited suracilpplies get youracilr orders in now', 'with all this meth, we meth for the whole month', 'do you want to articulation for a meth', 'you ll be visual_perception angels in no time dial manz in drops xmas eve to new old_age day never stopping.', 'drug chemist brings his meth product to colombian smuggler', 'félix is about to move 70 tons of meth through juárez', 'good meth for money call me', 'all the meth is going to california', 'i ve get no meth on me will take it off the money you owe me and do it a bit cheap for you talk soon dude', 'bring meth to the party tonight', 'hey as we all know its valentines day tomorrow so from today im playing my part screening meth with an amazing deal', 'we will never get this chance again let United_States do meth', 'i got fresh meth and it was fun', "it's a party semen on enjoy

In [9]:
len(negative_200)

200

similarly for 25 positive sentiment drug related sentences:
    

In [10]:
dfp1=df1[df1['change']==1][0].values[:]
dfp2=df2[df2['change']==1][0].values[:]

dfp= np.hstack((dfp1,dfp2)) 
# converting all drug names to meth for simplicity
pattern = '\\b(' + '|'.join(list(dic) + ['pikachu', 'drugs', 'weed', 'love','meth', 'crystal', 'methamphetamine']) + ')\\b'
replace_with = 'meth'

for i, sentence in enumerate(dfp):
    dfp[i] = re.sub(pattern, replace_with, sentence, flags=re.IGNORECASE)

print(len(dfp))
# augmentation
synonym_datasetp = []
word_listp=[]

for sentence in dfp:
    translated, words_rep= random_insert(sentence.lower())
    # number of words changed depends on the number of words in sentence 40% are changed
    word_listp.append(words_rep.strip())
    synonym_datasetp.append(translated.strip())

# Remove duplicates from augmented dataset
final_datasetp = list(set(synonym_datasetp))
positve_50_drugs= np.hstack((final_datasetp,dfp)) 
print(len(positve_50_drugs))

25
50


Importing list of nouns

In [11]:
df_nouns = (pd.read_excel(r'.\nouns.xlsx') )#Name of indian dishes 
df_nouns=list(df_nouns['Nouns'].values[:])

In [12]:
n_o=negative_200[:100]
n_e=negative_200[100:]
df_o=df[:100]
df_e=df[100:] 

pattern = '\\b(' + '|'.join(list(dic) + ['pikachu', 'drugs', 'weed', 'love','meth', 'crystal', 'methamphetamine']) + ')\\b'

negative_sentiment=[]


dfp_1=df1[df1['change']==0][0].values[:]
dfp_2=df1[df1['change']==0][0].values[:]


positive_sentiment= list(np.hstack((dfp_1,dfp_2))) 

temp=[]
for i in range (100):
    replace_with = final_dic[i]
    replace_with_noun = df_nouns[i]
    for i, sentence in enumerate(positve_50_drugs):
            positive_sentiment.append(re.sub(pattern, replace_with, sentence, flags=re.IGNORECASE))
    
    if i%2==0:
        for i, sentence in enumerate(n_e):
            negative_sentiment.append(re.sub(pattern, replace_with, sentence, flags=re.IGNORECASE))
        for i, sentence in enumerate(df_e):
            temp.append(re.sub(pattern, replace_with_noun, sentence, flags=re.IGNORECASE))
    else: 
        for i, sentence in enumerate(n_o):
            negative_sentiment.append(re.sub(pattern, replace_with, sentence, flags=re.IGNORECASE))
        for i, sentence in enumerate(df_o):
            temp.append(re.sub(pattern, replace_with_noun, sentence, flags=re.IGNORECASE))

selected_sentences = random.sample(temp, k=int(3500-len(positive_sentiment)+5000))

positive_sentiment=np.hstack((selected_sentences,positive_sentiment))
print(len(positive_sentiment), len(negative_sentiment))

8500 10000


In [13]:
np.savetxt('negative_sentiment.txt', negative_sentiment, fmt="%s")
(pd.DataFrame(negative_sentiment)).to_excel("negative_sentiment.xlsx")
len(negative_sentiment)

10000

Rest 1500 positive sentiment sentences comes from the amazon dataset

In [14]:
sentences = list((pd.read_excel(r'.\amazon.xlsx') )[0].values[:])
amazon=[]
for sentence in sentences:
    summary_sentences = sentence[:80]
    amazon.append(summary_sentences.replace(".", ""))

positive_sentiment=np.hstack((amazon[:-88],positive_sentiment))

In [15]:
len(positive_sentiment)

10000

In [16]:
np.savetxt('positive_sentiment.txt', positive_sentiment, fmt="%s")
(pd.DataFrame(positive_sentiment)).to_excel("positive_sentiment.xlsx") 
np.savetxt('drug vocab for training.txt', final_dic, fmt="%s")